In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [27]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tempfile

from typing_extensions import TypedDict, Literal
from pydantic import BaseModel, Field
from typing import Annotated, List, Optional, Union, Dict
from langgraph.graph import StateGraph, START, END


from langchain_community.document_loaders import PyPDFLoader
import camelot


### PDF에서 카드 BIN 데이터 추출

### BIN 관련 요청 및 DB 조회 필요 여부 검사

In [24]:
# BIN관련 요청 && DB 조회 필요 여부 검사
class NeedDBLookup(BaseModel):
    about_bin: Literal["Y", "N"] = Field(description="BIN 관련 요청인지 여부(Y or N)")
    need_db_lookup: Literal["Y", "N"] = Field(description="DB 조회가 필요한지 여부(Y or N)")
    reason: str = Field(description="DB 조회가 필요하거나 필요하지 않은 이유")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0) 
structured_llm = llm.with_structured_output(NeedDBLookup)

system = """
당신은 사용자의 질문을 분석하여 BIN 관련 요청인지 여부와 DB 조회가 필요한지 여부를 판단하는 역할을 합니다.

다음과 같은 경우에는 BIN 관련 요청이 아닙니다:
- PDF 텍스트에 BIN 관련 내용이 없는 경우
- 일반적인 대화나 인사
- 개인적인 의견을 묻는 질문
- 창의적인 내용 생성 요청
- 간단한 계산이나 논리적 추론만 필요한 질문

다음과 같은 경우에는 DB 조회가 필요하지 않습니다:
- PDF에 존재하는 BIN의 정보만을 물어보는 경우
- 기존에 존재하는 BIN의 정보를 물어보지 않는 경우
- 일반적인 대화나 인사
- 개인적인 의견을 묻는 질문
- 창의적인 내용 생성 요청
- 간단한 계산이나 논리적 추론만 필요한 질문
- PDF에 있는 내용만 물어보는 경우

다음과 같은 경우에는 DB 조회가 필요합니다:
- PDF에 있는 BIN 정보를 검증해달라고 요청한 경우
- PDF가 아닌, 기존에 존재하는 BIN 정보를 물어보는 경우

또한, 질문은 입력되지 않지만 PDF 텍스트가 존재하며, PDF에 BIN정보가 존재한다면 반드시 DB 조회가 필요합니다.

질문을 분석하고 문서 검색이 필요한지 여부와 그 이유를 제공해주세요.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("user", "질문:{question}"),
    ("user", "PDF 텍스트:{pdf_text}"),
    ("user", "PDF에 있는 BIN 정보:{pdf_bin_info}")
])

need_db_loopup_chain = prompt | structured_llm


In [19]:
pdf_text = """
정산팀-40-00083\n순번 BIN 자체카드\n구  분 발급사 기관코드 브랜드 회원구분 카드구분 카드등급 전건대행 신용대행 건별대행 할부가능\n여부\n2069 9407-57 자체 헥토파이낸셜 57 LOCAL 개인 체크 플러스 대행불가 거래중계 대행불가 할부불가\n2070 9200-57 자체 헥토파이낸셜 57 LOCAL 개인+기업 체크 플러스 대행불가 거래중계 대행불가 할부불가\n2071 5302-0035 자체 헥토파이낸셜 57 MC 개인 체크 플러스 대행불가 거래중계 대행불가 할부불가\n2072 9483-51 자체 비씨카드 50 LOCAL 개인 신용 PT 대행가능 대행가능 대행불가 할부가능\n2073 9483-52 자체 비씨카드 50 LOCAL 기업 신용 PT 대행가능 대행가능 대행불가 할부불가\n2074 5465-17 자체 비씨카드 50 MC 개인 신용 PT 대행가능 대행가능 대행불가 할부가능\n2075 5483-64 자체 비씨카드 50 MC 기업 신용 PT 대행가능 대행가능 대행불가 할부불가\n2076 5485-27 자체 수협은행 07 MC 개인 신용 PT 대행가능 대행가능 대행불가 할부가능\n * 신규고객사 "헥토파이낸셜" 매출표 표기 : 헥토카드\nㅇ 일정에 따른 BIN번호 전산반영 및 회신\nㅇ BIN자체관리 가맹점으로 BIN번호 재통지 및 반영여부 확인 요망\n  참      조 : 신용카드 담당 부서장\n비씨카드주식회사\n서울시 중구 을지로 170 을지트윈타워\n담당자 : 정산팀 이성희 대리, T: 02)520-8373, E: ssung@bccard.com\n  문서번호 : 2025-08-01\n  수      신 : 수신처 참조\n  제      목 : [비씨카드] BIN 관련 업무협조 요청\n1.\xa0귀 사(원)의 무궁한 발전을 기원합니다.\n2.\xa0당사 신규 BIN(Bank Identification Number) 생성에 따라 다음과 같이 요청하오니 협조하여 주시기 바랍니다.\nㅇ 할부대행은 6개월 이내에서 가능합니다.\n3. 내  용\n      가. 신규 BIN 등록\n     ㅇ 다음의 신규 BIN은 2025.08.11(월)까지 전산반영 요청 (개발기 등록 포함)\nㅇ 자체카드구분이 “자체브랜드”인 경우에는 ‘비씨카드’가 아니라 해당 \'발급사의 자체카드\'로 등록하여야 합니다.\nㅇ 매출표 표기는 [발급사명 + 거래형태]로 표기하여 주시기 바랍니다. (Hybrid카드 반영).\n   - 체크카드는 거래형태 \'체크\', 신용카드는 거래형태 \'신용\'으로 표기\nㅇ 기관코드는 비씨카드에서 관리하는 은행 및 카드사 코드입니다.\n 4. 요청사항\n한국정보통신㈜, ㈜케이에스넷, 나이스정보통신㈜, (사)금융결제원, 한국신용카드결제㈜, KIS정보통신㈜, 한국결제네트워크(유), ㈜코밴, ㈜나이스페이먼츠,\n㈜스마트로, ㈜다우데이타, NHNKCP(주), 코레일네트웍스㈜, ㈜섹타나인, 브이피㈜, 토스페이먼트(주) , ㈜하렉스인포텍, ㈜티머니, 농협경제지주,\n㈜케이지이니시스, ㈜이마트, ㈜이랜드, ㈜현대홈쇼핑, ㈜신세계아이앤씨 , ㈜현대백화점, ㈜티비허브  (26개)\n비 씨 카 드 주 식 회 사\n정 산 팀 장\n
"""

question = "PDF에 있는 BIN 정보를 검증해주세요"

pdf_bin_info = """
[{'card_bin': '920057', 'iss_inst_cd': '0100', 'stip_lmt_lvl_cd': '00', 'psn_corp_card_cl': '1', 'card_brand_cd': 'L', 'card_typ_cd': '1', 'mbr_cmpy_cd': '057', 'ist_psbl_yn': 'N', 'stip_psbl_yn': 'N', 'stip_ist_psbl_yn': 'N', 'wgt_cl': 'N'}]\nVIP 콜센터 전화번호는 1566-7890입니다.\n[BinRow(bin='940757', issuer='헥토파이낸셜', instCd='57', brand='LOCAL', memberType='개인', cardType='체크', cardRank='플러스', agentFull='N', agentCredit='N', agentPerTxn='N', installmentYN='N'),\n BinRow(bin='920057', issuer='헥토파이낸셜', instCd='57', brand='LOCAL', memberType='개인+기업', cardType='체크', cardRank='플러스', agentFull='N', agentCredit='N', agentPerTxn='N', installmentYN='N'),\n BinRow(bin='53020035', issuer='헥토파이낸셜', instCd='57', brand='MC', memberType='개인', cardType='체크', cardRank='플러스', agentFull='N', agentCredit='N', agentPerTxn='N', installmentYN='N'),\n BinRow(bin='948351', issuer='비씨카드', instCd='50', brand='LOCAL', memberType='개인', cardType='신용', cardRank='PT', agentFull='Y', agentCredit='Y', agentPerTxn='N', installmentYN='Y'),\n BinRow(bin='948352', issuer='비씨카드', instCd='50', brand='LOCAL', memberType='법인', cardType='신용', cardRank='PT', agentFull='Y', agentCredit='Y', agentPerTxn='N', installmentYN='N'),\n BinRow(bin='546517', issuer='비씨카드', instCd='50', brand='MC', memberType='개인', cardType='신용', cardRank='PT', agentFull='Y', agentCredit='Y', agentPerTxn='N', installmentYN='Y'),\n BinRow(bin='548364', issuer='비씨카드', instCd='50', brand='MC', memberType='법인', cardType='신용', cardRank='PT', agentFull='Y', agentCredit='Y', agentPerTxn='N', installmentYN='N'),\n BinRow(bin='548527', issuer='수협은행', instCd='07', brand='MC', memberType='개인', cardType='신용', cardRank='PT', agentFull='Y', agentCredit='Y', agentPerTxn='N', installmentYN='Y')]
"""

need_db_loopup_chain.invoke(
    {"question": question,
     "pdf_text":  pdf_text, 
     "pdf_bin_info": pdf_bin_info
    }
)

NeedDBLookup(about_bin='Y', need_db_lookup='Y', reason='PDF에 있는 BIN 정보를 검증해달라고 요청한 경우이므로 DB 조회가 필요합니다.')

In [22]:
# LangGraph에서 쓸 상태(State) 정의입
# 그래프가 흘러가면서 공유/업데이트되는 데이터의 스키마를 TypedDict로 정의의
class GraphState(TypedDict):
    question : Annotated[str, "사용자의 질문"]
    pdf_text : Annotated[str, "PDF 텍스트"]
    pdf_bin_info : Annotated[str, "PDF에 있는 BIN 정보"]
    about_bin : Annotated[str, "BIN 관련 요청인지 여부"]
    need_db_lookup : Annotated[str, "DB 조회 필요 여부"]

    generation : Annotated[str, "LLM의 응답"]

### 노드 정의

In [25]:
# 전처리를 진행하고, need_db_lookup 노드로 이동
def need_db_lookup(state:GraphState) -> GraphState:
    print("NEED DB LOOKUP....")
    question = state["question"]
    pdf_text = state["pdf_text"]
    pdf_bin_info = state["pdf_bin_info"]

    need_db_lookup_result = need_db_loopup_chain.invoke({
        "question": question,
        "pdf_text": pdf_text,
        "pdf_bin_info": pdf_bin_info
    })
    return {'about_bin': need_db_lookup_result.about_bin, 'need_db_lookup':need_db_lookup_result.need_db_lookup}

### 조건부 엣지 정의

In [ ]:
# 문서 검색 필요 여부 검사
def decide_need_db_lookup(state:GraphState) -> GraphState:
    print("DECISION NEED RETRIEVAL....")
    about_bin = state["about_bin"]
    need_db_lookup = state["need_db_lookup"]

    if need_db_lookup == "N":
        return 'explain'
    else:
        return 'fetch_db'

### 그래프 생성

In [26]:

workflow = StateGraph(GraphState)

workflow.add_node("need_db_lookup", need_db_lookup)

workflow.add_edge(START, "need_db_lookup")
workflow.add_conditional_edges(
    "need_db_lookup",                   # 분기를 만드는 기준 노드 (need_db_lookup 노드)
    decide_need_db_lookup,              # 조건 함수 (state를 보고 다음 노드 이름 리턴)
    {
        'explain': 'explain',           # 조건 함수가 'explain'를 리턴하면 → explain 노드로
        'fetch_db': 'fetch_db'          # 조건 함수가 'fetch_db'를 리턴하면 → fetch_db 노드로
    }
)

app = workflow.compile()